<a href="https://colab.research.google.com/github/DavidSaruni/Chat-Bot/blob/main/MyChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Requirements**

In [ ]:
# Things we need for NLP
import nltk #Natural Language Processing Tool Kit
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Things we need for Tensorflow
import tensorflow as tf
import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
! pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 24.6 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=1f36e7927ba177de8c1c42f1090b564ad7300e5d4613512ced3259e99f0137a8
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


**Google functionality to import file**

In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{"intents":[\r\n    {\r\n        "tag": "greeting",\r\n        "patterns": ["Hi", "Hey", "Hello", "How are you?", "Good day", "Is anyone there?"],\r\n        "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help you?"],\r\n        "context_set": ""\r\n    },\r\n    {\r\n        "tag": "goodbye",\r\n        "patterns": ["Bye", "Goodbye", "See you later"],\r\n        "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n    },\r\n    {\r\n        "tag": "thanks",\r\n        "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n        "responses": ["Happy to help", "You\'re welcome always", "Anytime", "My pleasure"]   \r\n    },\r\n    {\r\n        "tag": "hours",\r\n        "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n        "responses": ["We\'re open every week day 8am-5pm", "Our hours are 8am-5pm for week days"]\r\n    }\r\n

**Intents File**

In [ ]:
# Import our chatbot intents file
import json
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [ ]:
from nltk.corpus.reader import documents
words = []
classes = []
documents = []
ignore_words = ['?']

# looping through each sentence in our intents patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenization
    w = nltk.word_tokenize(pattern)
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, intent['tag']))
    # add to our classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])


# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

15 documents
4 classes ['goodbye', 'greeting', 'hours', 'thanks']
24 unique stemmed words ["'s", 'anyon', 'ar', 'bye', 'day', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'hour', 'how', 'is', 'lat', 'op', 'see', 'thank', 'that', 'ther', 'what', 'when', 'yo', 'you']


**Training**
Transforming them from document of words into tensors of numbers

In [ ]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # create list for tokenized words for the patterns
  pattern_words = doc[0]
  # stem each word 
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create our bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  # output is a '0' for each tag and '1' for current tag
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

# shuffle our features to numpy array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


**Build Neural Network**

In [ ]:
# reset underlying graph data
#tf.reset_default_graph()
from tensorflow.python.framework import ops
ops.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, batch_size=8, show_metric=True, n_epoch=1000)
model.save('model.chatbot')

Training Step: 1999  | total loss: 0.07587 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.07587 - acc: 0.9988 -- iter: 08/15
Training Step: 2000  | total loss: 0.07511 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.07511 - acc: 0.9989 -- iter: 15/15
--


In [ ]:
def clean_up(sentence):
  # tokenize the pattern
  sentence_words = nltk.word_tokenize(sentence)
  # stem each word
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words


In [ ]:
from nltk.inference.discourse import show_cfg
# return bag of words array
def bow(sentence, words, show_details=False):
  # tokenize the pattern
  sentence_words = clean_up(sentence)
  # bag of words
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
        if show_details:
          print("Found in bag" % w)
  return (np.array(bag))

In [ ]:
s = bow("Hi", words)
print(s)
print(classes)

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
['goodbye', 'greeting', 'hours', 'thanks']


In [ ]:
print(model.predict([s]))

[[1.8426716e-02 9.8154896e-01 8.8160978e-06 1.5526537e-05]]


In [ ]:
# save all of your data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open("training_data", "wb"))

In [ ]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [ ]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'thanks']


In [ ]:
# load our saved model
model.load('model.chatbot')

NotFoundError: ignored

# New Section